In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import LabelEncoder
from category_encoders.target_encoder import TargetEncoder

train_df =pd.read_csv('data/modified_train.csv')
test_df = pd.read_csv('data/modified_test.csv')


In [2]:
light_df = pd.read_csv('data/external_open/대구 보안등 정보.csv', encoding='cp949')[['설치개수', '소재지지번주소']]

location_pattern = r'(\S+) (\S+) (\S+) (\S+)'

light_df[['도시', '구', '동', '번지']] = light_df['소재지지번주소'].str.extract(location_pattern)
light_df = light_df.drop(columns=['소재지지번주소', '번지'])

light_df = light_df.groupby(['도시', '구', '동']).sum().reset_index()
light_df.reset_index(inplace=True, drop=True)

C:\Users\HONG\AppData\Local\Temp\ipykernel_11972\362901134.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  light_df = pd.read_csv('data/external_open/대구 보안등 정보.csv', encoding='cp949')[['설치개수', '소재지지번주소']]


In [3]:
parking_df = pd.read_csv('data/external_open/대구 주차장 정보.csv', encoding='cp949')[['소재지지번주소', '급지구분']]
parking_df = pd.get_dummies(parking_df, columns=['급지구분'])

location_pattern = r'(\S+) (\S+) (\S+) (\S+)'

parking_df[['도시', '구', '동', '번지']] = parking_df['소재지지번주소'].str.extract(location_pattern)
parking_df = parking_df.drop(columns=['소재지지번주소', '번지'])

parking_df = parking_df.groupby(['도시', '구', '동']).sum().reset_index()
parking_df.reset_index(inplace=True, drop=True)

In [4]:
child_area_df = pd.read_csv('data/external_open/대구 어린이 보호 구역 정보.csv', encoding='cp949').drop_duplicates()[['소재지지번주소']]
child_area_df['cnt'] = 1

location_pattern = r'(\S+) (\S+) (\S+) (\S+)'

child_area_df[['도시', '구', '동', '번지']] = child_area_df['소재지지번주소'].str.extract(location_pattern)
child_area_df = child_area_df.drop(columns=['소재지지번주소', '번지'])

child_area_df = child_area_df.groupby(['도시', '구', '동']).sum().reset_index()
child_area_df.reset_index(inplace=True, drop=True)

In [5]:
# train_df와 test_df에, light_df와 child_area_df, parking_df를 merge하세요.
train_df = pd.merge(train_df, light_df, how='left', on=['도시', '구', '동'])
train_df = pd.merge(train_df, child_area_df, how='left', on=['도시', '구', '동'])
train_df = pd.merge(train_df, parking_df, how='left', on=['도시', '구', '동'])

test_df = pd.merge(test_df, light_df, how='left', on=['도시', '구', '동'])
test_df = pd.merge(test_df, child_area_df, how='left', on=['도시', '구', '동'])
test_df = pd.merge(test_df, parking_df, how='left', on=['도시', '구', '동'])

In [6]:
train_df['설치개수'].mean()

881.9394271664008

In [7]:
train_df['cnt'] = train_df['cnt'].fillna(train_df['cnt'].mean())
train_df['설치개수'] = train_df['설치개수'].fillna(train_df['설치개수'].mean())

train_df['급지구분_1'] = train_df['급지구분_1'].fillna(train_df['급지구분_1'].mean())
train_df['급지구분_2'] = train_df['급지구분_2'].fillna(train_df['급지구분_2'].mean())
train_df['급지구분_3'] = train_df['급지구분_3'].fillna(train_df['급지구분_3'].mean())

test_df['cnt'] = test_df['cnt'].fillna(test_df['cnt'].mean())
test_df['설치개수'] = test_df['설치개수'].fillna(test_df['설치개수'].mean())

test_df['급지구분_1'] = test_df['급지구분_1'].fillna(test_df['급지구분_1'].mean())
test_df['급지구분_2'] = test_df['급지구분_2'].fillna(test_df['급지구분_2'].mean())
test_df['급지구분_3'] = test_df['급지구분_3'].fillna(test_df['급지구분_3'].mean())

In [8]:
test_x = test_df.drop(columns=['ID']).copy()
train_x = train_df[test_x.columns].copy()
train_y = train_df['ECLO'].copy()



In [9]:
categorical_features = list(train_x.dtypes[train_x.dtypes == "object"].index)
# 추출된 문자열 변수 확인
display(categorical_features)

for i in categorical_features:
    te = TargetEncoder(cols=[i])
    train_x[i] = te.fit_transform(train_x[i], train_y)
    test_x[i] = te.transform(test_x[i])
    

['요일', '기상상태', '노면상태', '사고유형', '도시', '구', '동', '도로형태1', '도로형태2']

In [10]:
train_x = train_x.drop(columns='Unnamed: 0')
test_x = test_x.drop(columns='Unnamed: 0')

display(train_x)
display(test_x)

,요일,기상상태,노면상태,사고유형,연,월,일,시간,도시,구,동,도로형태1,도로형태2,설치개수,cnt,급지구분_1,급지구분_2,급지구분_3
0,4.627926,4.712888,4.712878,3.817650,2019,1,1,0,4.726704,4.541610,4.282449,4.671841,4.599599,391.000000,2.000000,11.000000,0.00000,0.000000
1,4.627926,4.779150,4.712878,3.817650,2019,1,1,0,4.726704,4.618441,4.738938,4.671841,4.599599,932.000000,11.672426,0.000000,1.00000,3.000000
2,4.627926,4.712888,4.712878,3.817650,2019,1,1,1,4.726704,4.727300,4.842715,4.671841,4.599599,473.000000,5.000000,2.549961,4.50121,3.164217
3,4.627926,4.712888,4.712878,4.944597,2019,1,1,2,4.726704,4.687669,4.208920,4.671841,4.599599,534.000000,11.000000,0.000000,9.00000,5.000000
4,4.627926,4.712888,4.712878,4.944597,2019,1,1,4,4.726704,4.889534,4.549091,4.671841,4.599599,2057.000000,11.672426,0.000000,1.00000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39604,4.600906,4.712888,4.712878,4.944597,2021,12,31,19,4.726704,4.727300,4.913420,4.882281,5.006142,881.939427,1.000000,2.549961,4.50121,3.164217
39605,4.600906,4.712888,4.712878,4.944597,2021,12,31,19,4.726704,4.618441,4.293963,4.671841,4.599599,843.000000,11.672426,0.000000,0.00000,5.000000
39606,4.600906,4.712888,4.712878,4.944597,2021,12,31,21,4.726704,4.618441,4.648867,4.882281,5.006142,164.000000,11.672426,0.000000,1.00000,0.000000
39607,4.600906,4.712888,4.712878,4.944597,2021,12,31,22,4.726704,4.618441,4.672002,4.065962,4.599599,210.000000,11.672426,0.000000,0.00000,1.000000


,요일,기상상태,노면상태,사고유형,연,월,일,시간,도시,구,동,도로형태1,도로형태2,설치개수,cnt,급지구분_1,급지구분_2,급지구분_3
0,4.920811,4.712888,4.712878,3.817650,2022,1,1,1,4.726704,4.727300,4.881657,4.882281,5.006142,700.000000,5.000000,2.503929,4.49275,3.18539
1,4.920811,4.712888,4.712878,3.817650,2022,1,1,1,4.726704,4.727300,4.563008,4.671841,4.599599,852.041626,10.000000,0.000000,0.00000,2.00000
2,4.920811,4.712888,4.712878,4.944597,2022,1,1,4,4.726704,4.727300,4.945578,4.882281,5.006142,852.041626,1.000000,2.503929,4.49275,3.18539
3,4.920811,4.712888,4.712878,4.944597,2022,1,1,4,4.726704,4.727300,4.438172,4.671841,4.599599,852.041626,7.000000,0.000000,2.00000,1.00000
4,4.920811,4.712888,4.712878,4.944597,2022,1,1,6,4.726704,4.618441,4.738938,4.882281,5.006142,932.000000,11.347384,0.000000,1.00000,3.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10958,4.920811,4.712888,4.712878,4.944597,2022,12,31,18,4.726704,4.623681,4.492281,4.671841,6.555670,5377.000000,26.000000,20.000000,1.00000,0.00000
10959,4.920811,4.712888,4.712878,4.944597,2022,12,31,18,4.726704,4.727300,4.928144,4.671841,4.599599,852.041626,5.000000,2.503929,4.49275,3.18539
10960,4.920811,4.712888,4.712878,4.944597,2022,12,31,20,4.726704,4.727300,5.355036,4.671841,4.599599,852.041626,11.347384,2.503929,4.49275,3.18539
10961,4.920811,4.712888,4.712878,4.944597,2022,12,31,20,4.726704,4.727300,5.079498,4.882281,4.863035,852.041626,7.000000,2.503929,4.49275,3.18539


In [29]:
from sklearn.model_selection import KFold, StratifiedKFold
from catboost import CatBoostRegressor, Pool

train_preds = np.zeros(len(train_x))
test_preds = np.zeros(len(test_x))

# skf = KFold(n_splits=5, shuffle=True, random_state=42)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for i, (train_index, valid_index) in enumerate(skf.split(train_x, train_y)):
    dtrain = Pool(data=train_x.values[train_index], label=train_y.values[train_index])
    dvalid = Pool(data=train_x.values[valid_index], label=train_y.values[valid_index])
    bst = CatBoostRegressor(
        iterations=1000,
        objective="RMSE",
        learning_rate=1e-2,
        l2_leaf_reg=6,
        depth=4,
        random_seed=42,
        subsample=0.7,
        bagging_temperature=0.23,
        od_type="Iter",
        grow_policy='Depthwise'
    )
    bst.fit(X=dtrain, eval_set=dvalid)
    test_preds += bst.predict(Pool(test_x.values)) / skf.n_splits


sample_submission = pd.read_csv("data/sample_submission.csv")
baseline_submission = sample_submission.copy()
baseline_submission["ECLO"] = test_preds.astype(int)
baseline_submission.to_csv("result/cb_te_fillna.csv", index=False)

c:\Users\HONG\miniconda3\envs\daegu\lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


0:	learn: 3.2193251	test: 3.1532477	best: 3.1532477 (0)	total: 6.94ms	remaining: 6.93s
1:	learn: 3.2182717	test: 3.1523329	best: 3.1523329 (1)	total: 11.7ms	remaining: 5.84s
2:	learn: 3.2173615	test: 3.1514921	best: 3.1514921 (2)	total: 17.5ms	remaining: 5.82s
3:	learn: 3.2164250	test: 3.1506292	best: 3.1506292 (3)	total: 22.6ms	remaining: 5.62s
4:	learn: 3.2155790	test: 3.1498576	best: 3.1498576 (4)	total: 27.3ms	remaining: 5.43s
5:	learn: 3.2146391	test: 3.1489895	best: 3.1489895 (5)	total: 31.1ms	remaining: 5.15s
6:	learn: 3.2137940	test: 3.1482577	best: 3.1482577 (6)	total: 34.8ms	remaining: 4.94s
7:	learn: 3.2129462	test: 3.1474393	best: 3.1474393 (7)	total: 38.6ms	remaining: 4.78s
8:	learn: 3.2120387	test: 3.1466170	best: 3.1466170 (8)	total: 43.3ms	remaining: 4.77s
9:	learn: 3.2111980	test: 3.1458698	best: 3.1458698 (9)	total: 47.1ms	remaining: 4.67s
10:	learn: 3.2102662	test: 3.1450800	best: 3.1450800 (10)	total: 50.9ms	remaining: 4.58s
11:	learn: 3.2093468	test: 3.1442794	best

In [16]:
import lightgbm as lgb
# lightgbm custom metric 적용하기

def RMSLE_LGBM(y, t):
    t = t.get_label()
    # print(y.shape, t.shape)
    log_y = np.log1p(y)
    log_t = np.log1p(t)
    loss = np.sqrt(np.mean((log_y - log_t)**2))
    return 'rmsle', loss, False


params = {
    # 'objective': 'root_mean_squared_error',
    "boosting_type": "gbdt",
    "objective": "regression",
    "learning_rate": 0.1,
    "seed": 42,
    "lambda_l1": 5,
    "lambda_l2": 5,
    "max_depth": 3,
    "num_leaves": 4,
    "force_col_wise": True,
    "nthread": 24,
    "bagging_fraction": 0.6,
    "feature_fraction": 0.5,
}

train_preds = np.zeros(len(train_x))
test_preds = np.zeros(len(test_x))


skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# skf = KFold(n_splits=5, shuffle=True, random_state=42)
for i, (train_index, valid_index) in enumerate(skf.split(train_x, train_y)):
    # print(train_x.values[train_index].shape, train_x.values[train_index].shape, '@@@@@@')
    # print(train_x.values[valid_index].shape, train_y.values[valid_index].shape, '######')
    
    dtrain = lgb.Dataset(train_x.values[train_index], label=train_y.values[train_index], free_raw_data=False)
    dvalid = lgb.Dataset(train_x.values[valid_index], label=train_y.values[valid_index], free_raw_data=False)

    bst = lgb.train(params, dtrain, num_boost_round=1000, valid_sets=[dvalid], feval=RMSLE_LGBM)
    # train_preds[valid_index] = bst.predict(dvalid)
    # test_preds += bst.predict(lgb.Dataset(test_x)) / skf.n_splits
    test_preds += bst.predict(test_x.values) / skf.n_splits

# submit
sample_submission = pd.read_csv("data/sample_submission.csv")
baseline_submission = sample_submission.copy()
baseline_submission["ECLO"] = test_preds.astype(int)
baseline_submission.to_csv("result/lgbm_te_strkf.csv", index=False)

baseline_submission['ECLO'].value_counts()


c:\Users\HONG\miniconda3\envs\daegu\lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


[LightGBM] [Info] Total Bins 515
[LightGBM] [Info] Number of data points in the train set: 31687, number of used features: 17
[LightGBM] [Info] Start training from score 4.727806
[LightGBM] [Info] Total Bins 512
[LightGBM] [Info] Number of data points in the train set: 31687, number of used features: 17
[LightGBM] [Info] Start training from score 4.725881
[LightGBM] [Info] Total Bins 513
[LightGBM] [Info] Number of data points in the train set: 31687, number of used features: 17
[LightGBM] [Info] Start training from score 4.726607
[LightGBM] [Info] Total Bins 512
[LightGBM] [Info] Number of data points in the train set: 31687, number of used features: 17
[LightGBM] [Info] Start training from score 4.725313
[LightGBM] [Info] Total Bins 514
[LightGBM] [Info] Number of data points in the train set: 31688, number of used features: 17
[LightGBM] [Info] Start training from score 4.727910


ECLO
4     6350
5     2162
3     2152
6      203
2       37
7       34
9       11
8        6
10       6
11       2
Name: count, dtype: int64

In [14]:
a = pd.read_csv('result/cb_te_strfk.csv')
a['ECLO'].value_counts()

ECLO
4     6649
5     2234
3     1954
6      103
7       19
8        3
12       1
Name: count, dtype: int64

In [ ]:
# catboost bestTest = 3.109126901 >> LB:0.4303
# fillna : mean,
        # iterations=1000,
#         objective="RMSE",
#         learning_rate=0.01,
#         l2_leaf_reg=6,
#         depth=4,
#         random_seed=42,
#         subsample=0.7,
#         bagging_temperature=0.23,
#         od_type="Iter",, StratifiedKFold

bestTest = 3.10905465   /// 9
depthwise = 3.1086